In [11]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

In [2]:
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


Разделим наши данные на тренировочную и тестовую выборки

In [3]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

К полям:
- gender, cholesterol применим OHE-кодирование
- age, height, weight, ap_hi, ap_lo - standardScaler
- gluc, smoke, alco, active - оставим пока как есть

In [4]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

Теперь объединим все наши трансформеры с помощью FeatureUnion

In [5]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

Добавим классификатор и запустим кросс-валидацию

In [12]:
classifier_log_reg = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])
classifier_boost = Pipeline([
    ('features',feats),
    ('classifier', GradientBoostingClassifier(random_state = 42)),
])
classifier_forest = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

#запустим кросс-валидацию
cv_scores_log_reg = cross_val_score(classifier_log_reg, X_train, y_train, cv=16, scoring='roc_auc')
cv_score_log_reg = np.mean(cv_scores_log_reg)
cv_score_std_log_reg = np.std(cv_scores_log_reg)
print('CV score is {}+-{}'.format(cv_score_log_reg, cv_score_std_log_reg))

#обучим пайплайн на всем тренировочном датасете
classifier_log_reg.fit(X_train, y_train)
y_score_log_reg = classifier_log_reg.predict_proba(X_test)[:, 1]

CV score is 0.7867401104915408+-0.00852135511666111


In [13]:
#запустим кросс-валидацию для второй модели
cv_scores_boost = cross_val_score(classifier_boost, X_train, y_train, cv=16, scoring='roc_auc')
cv_score_boost = np.mean(cv_scores_boost)
cv_score_std_boost = np.std(cv_scores_boost)
print('CV score is {}+-{}'.format(cv_score_boost, cv_score_std_boost))

#обучим пайплайн для второй модели на всем тренировочном датасете
classifier_boost.fit(X_train, y_train)
y_score_boost = classifier_boost.predict_proba(X_test)[:, 1]

CV score is 0.8025124517417064+-0.007075372179995901


In [14]:
#запустим кросс-валидацию для третьей модели
cv_scores_forest = cross_val_score(classifier_forest, X_train, y_train, cv=16, scoring='roc_auc')
cv_score_forest = np.mean(cv_scores_forest)
cv_score_std_forest = np.std(cv_scores_forest)
print('CV score is {}+-{}'.format(cv_score_forest, cv_score_std_forest))

#обучим пайплайн для третьей модели на всем тренировочном датасете
classifier_forest.fit(X_train, y_train)
y_score_forest = classifier_forest.predict_proba(X_test)[:, 1]

CV score is 0.7734501681056019+-0.007171140345435727


Посчитаем precision/recall/f_score

In [27]:
b=1
precision_log_reg, recall_log_reg, thresholds_log_reg = precision_recall_curve(y_test.values, y_score_log_reg)
fscore_log_reg = (1+b**2)*(precision_log_reg * recall_log_reg) / (b**2*precision_log_reg + recall_log_reg)
ix_log_reg = np.argmax(fscore_log_reg)
print('LogReg\nBest Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_log_reg[ix], 
                                                                        fscore_log_reg[ix],
                                                                        precision_log_reg[ix],
                                                                        recall_log_reg[ix]))
cnf_matrix_log_reg = confusion_matrix(y_test, y_score_log_reg>thresholds_log_reg[ix])

LogReg
Best Threshold=0.068316, F-Score=0.663, Precision=0.497, Recall=0.996


In [28]:
b=1
precision_boost, recall_boost, thresholds_boost = precision_recall_curve(y_test.values, y_score_boost)
fscore_boost = (1+b**2)*(precision_boost * recall_boost) / (b**2*precision_boost + recall_boost)
ix_boost = np.argmax(fscore_boost)
print('Boost\nBest Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_boost[ix], 
                                                                        fscore_boost[ix],
                                                                        precision_boost[ix],
                                                                        recall_boost[ix]))
cnf_matrix_boost = confusion_matrix(y_test, y_score_boost>thresholds_boost[ix])

Boost
Best Threshold=0.077522, F-Score=0.667, Precision=0.500, Recall=1.000


In [29]:
b=1
precision_forest, recall_forest, thresholds_forest = precision_recall_curve(y_test.values, y_score_forest)
fscore_forest = (1+b**2)*(precision_forest * recall_forest) / (b**2*precision_forest + recall_forest)
ix_forest = np.argmax(fscore_forest)
print('Forest\nBest Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_forest[ix], 
                                                                        fscore_forest[ix],
                                                                        precision_forest[ix],
                                                                        recall_forest[ix]))
cnf_matrix_forest = confusion_matrix(y_test, y_score_forest>thresholds_forest[ix])

Forest
Best Threshold=0.350000, F-Score=0.719, Precision=0.643, Recall=0.816


In [26]:
from sklearn.metrics import roc_auc_score, log_loss
print("roc auc score LogReg: {}".format(roc_auc_score(y_true=y_test, y_score=y_score_log_reg)))
print("log loss score LogReg: {}".format(log_loss(y_true=y_test, y_pred=y_score_log_reg)))
print("roc auc score Boost: {}".format(roc_auc_score(y_true=y_test, y_score=y_score_boost)))
print("log loss score Boost: {}".format(log_loss(y_true=y_test, y_pred=y_score_boost)))
print("roc auc score Forest: {}".format(roc_auc_score(y_true=y_test, y_score=y_score_forest)))
print("log loss score Forest: {}".format(log_loss(y_true=y_test, y_pred=y_score_forest)))

roc auc score LogReg: 0.7840347790421852
log loss score LogReg: 0.5779604008230679
roc auc score Boost: 0.8026153641179974
log loss score Boost: 0.5397460438742135
roc auc score Forest: 0.7710366181802983
log loss score Forest: 0.5992984853728378


In [30]:
TN_log_reg = cnf_matrix_log_reg[0][0]
FN_log_reg = cnf_matrix_log_reg[1][0]
TP_log_reg = cnf_matrix_log_reg[1][1]
FP_log_reg = cnf_matrix_log_reg[0][1]
TPR_log_reg = TP_log_reg/(TP_log_reg+FN_log_reg)
FPR_log_reg = FP_log_reg/(FP_log_reg+TN_log_reg)
TNR_log_reg = TN_log_reg/(FP_log_reg+TN_log_reg)
TPR_log_reg, FPR_log_reg, TNR_log_reg

(0.9961981566820276, 0.9934240362811791, 0.006575963718820862)

In [31]:
TN_boost = cnf_matrix_boost[0][0]
FN_boost = cnf_matrix_boost[1][0]
TP_boost = cnf_matrix_boost[1][1]
FP_boost = cnf_matrix_boost[0][1]
TPR_boost = TP_boost/(TP_boost+FN_boost)
FPR_boost = FP_boost/(FP_boost+TN_boost)
TNR_boost = TN_boost/(FP_boost+TN_boost)
TPR_boost, FPR_boost, TNR_boost

(0.9996543778801843, 0.9834467120181406, 0.01655328798185941)

In [32]:
TN_forest = cnf_matrix_forest[0][0]
FN_forest = cnf_matrix_forest[1][0]
TP_forest = cnf_matrix_forest[1][1]
FP_forest = cnf_matrix_forest[0][1]
TPR_forest = TP_forest/(TP_forest+FN_forest)
FPR_forest = FP_forest/(FP_forest+TN_forest)
TNR_forest = TN_forest/(FP_forest+TN_forest)
TPR_forest, FPR_forest, TNR_forest

(0.8084101382488479, 0.43412698412698414, 0.5658730158730159)

Первые две модели, на мойвзгляд достаточно много ошибаются, FPR около 1, поэтому считаю, что лучше справилась модель номер 3 - Random Forest, если, конечно, нет косяка в обучении. 